In [ ]:
WANDB_PROJECT = "analysis2"
ENTITY = 'course'
CLASSES = {i:c for i,c in enumerate(['ENAs', 'micronucleus', 'normal', 'other'])}
RAW_DATA_ARTIFACT = 'dataset_23062023'
PROCESSED_DATA_ARTIFACT = 'dataset_23062023_split'

# Configs

In [ ]:
# Installs
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Imports
import os
import wandb
import shutil
import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold

# Split

In [ ]:
# Init run and get the artifact
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="data_split", name='split')
raw_data_artifact = run.use_artifact(f'{RAW_DATA_ARTIFACT}:latest')
path = Path(raw_data_artifact.download())

In [ ]:
# Get the table and select relevant columns
orig_eda_table = raw_data_artifact.get("eda_table")

fnames = orig_eda_table.get_column('file_name')
groups = orig_eda_table.get_column('original_file')
y = orig_eda_table.get_column('label')

wandb:   2130 of 2130 files downloaded.  


In [ ]:
# Create df for the data
df = pd.DataFrame()
df['file_name'] = fnames
df['fold'] = -1

In [ ]:
# Create folds using group partition (by original image) and stratify partition (by label)
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, valid_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[valid_idxs, ['fold']] = i

In [ ]:
# Assign two folds for validation (20%)
df['split'] = 'train'
df.loc[df.fold == 0, ['split']] = 'valid'
df.loc[df.fold == 1, ['split']] = 'valid'
del df['fold']
df.split.value_counts()

train    852
valid    213
Name: split, dtype: int64

In [ ]:
# Save df
df.to_csv('data_split.csv', index=False)

In [ ]:
# Create new artifact and add data
processed_data_artifact = wandb.Artifact(PROCESSED_DATA_ARTIFACT, type="split_data")
processed_data_artifact.add_file('data_split.csv')
processed_data_artifact.add_dir(path)

wandb: Adding directory to artifact (./artifacts/dataset_23062023:v1)... Done. 1.7s


In [ ]:
# Create and join table
data_split_table = wandb.Table(dataframe=df[['file_name', 'split']])
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "file_name")
processed_data_artifact.add(join_table, "eda_table_data_split")

In [ ]:
# Log artifact to W&B and finish run
run.log_artifact(processed_data_artifact)
run.finish()